In [1]:
from pprint import pprint
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_qdrant import FastEmbedSparse
from qdrant_client.models import  SparseVector
from qdrant_client import QdrantClient, models
from langchain import PromptTemplate    

import time
# from langchain.chains import RunnableWithMessageHistory




In [2]:
import session_info
session_info.show()

C:\Users\quint\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\session_info\main.py:213: UserWarning: The '__version__' attribute is deprecated and will be removed in MarkupSafe 3.1. Use feature detection, or `importlib.metadata.version("markupsafe")`, instead.
  mod_version = _find_version(mod.__version__)


In [2]:
# Load environment variables from .env file
load_dotenv()

# Access OpenAI API Key
openai_api_key = os.getenv("OPENAI_API_KEY")

# Debug: Check if the API key is being correctly loaded
if openai_api_key is None or openai_api_key == "":
    print("OpenAI API Key not found! Please make sure it is set correctly in the .env file.")
else:
    print(f"OpenAI API Key Loaded: {openai_api_key[:4]}...")  # Only print part of the key for security reasons


llmName = ChatOpenAI(model="gpt-4o")
Chat_llm = ChatOpenAI(model="gpt-4o", temperature=1)



dense_embedding_model = OpenAIEmbeddings(
                api_key=openai_api_key,
                model="text-embedding-3-large"
            )
sparse_embedding_model = FastEmbedSparse(model_name="Qdrant/bm25")

OpenAI API Key Loaded: sk-p...


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

In [3]:
Clientpath = "VectorDB/CombinedData"
client = QdrantClient(path=Clientpath)
merged_collection_name = "merged vectors collection"

In [4]:
def merged_dense_for_RRF(user_question, limitN):
    dense_query_vector =  dense_embedding_model.embed_query(user_question)
    merged_dense = client.query_points(
        merged_collection_name,
        query=dense_query_vector,
        using="text-embedding-3-large",
        limit=limitN,
        with_payload=True, 
    )

    return merged_dense

def merged_sparse_for_RRF(user_question, limitN):
    sparse_query_vector = sparse_embedding_model.embed_query(user_question)
    merged_sparse = client.query_points(
        collection_name=merged_collection_name,
        query=SparseVector(indices=sparse_query_vector.indices, values=sparse_query_vector.values),  
        using="bm25",
        with_payload=True,  
        limit=limitN  
    )
    
    return merged_sparse



In [5]:

def reciprocal_rank_fusion(results1, results2, k=60):
    """Combine results using Reciprocal Rank Fusion (RRF)."""
    rrf_scores = {}

    # Process results
    for rank, scored_point in enumerate(results1.points, start=1):  
        doc_id = scored_point.id
        rrf_scores[doc_id] = rrf_scores.get(doc_id, 0) + 1 / (k + rank)

    for rank, scored_point in enumerate(results2.points, start=1):  
        doc_id = scored_point.id
        rrf_scores[doc_id] = rrf_scores.get(doc_id, 0) + 1 / (k + rank)

    # Sort by the combined RRF score
    sorted_results = sorted(rrf_scores.items(), key=lambda x: x[1], reverse=True)

    return sorted_results

def Fetch_chunks_back(fused_results):    
    fused_ids = [doc_id for doc_id, _ in fused_results]


    detailed_results = client.retrieve(
        collection_name=merged_collection_name,
        ids=fused_ids,
        with_payload=True,  # Include payload in the results
        with_vectors=False,  # Optional: Include vectors if needed
    )

    detailed_results_with_scores = []
    for result in detailed_results:
        # print(result)
        doc_id = result.id
        score = next((s for d, s in fused_results if d == doc_id), 0)
        detailed_results_with_scores.append({"ID":result.id, "rrf_score": score, "payload": result.payload} )
    return detailed_results_with_scores




In [6]:
# detailed_results = client.retrieve(
#     collection_name=merged_collection_name,
#     ids=["84970a49-031c-466e-a29b-3145e79d1ef3", "e94499bc-9696-4bb2-92fb-d8128e085c60"],
#     with_payload=True,  # Include payload in the results
#     )

# for x in detailed_results:
#     print(x.id)
#     print(x.payload)

# # pprint(detailed_results.payload)

In [7]:
# Scroll through the collection
all_nodes = []
scroll_cursor = None

while True:
    points, next_page_offset = client.scroll(
        collection_name=merged_collection_name,
        offset=scroll_cursor,  
        with_payload=True,    
        with_vectors=False,    
        limit=600              
    )

    all_nodes.extend(points)  
    
    # If no more points are available, break
    if next_page_offset is None:
        break

    # Update the cursor
    scroll_cursor = next_page_offset



In [8]:
pprint(all_nodes)

[Record(id='00d0c2ca-9bf8-4883-b9c5-1aae3e34f041', payload={'content': '9.1.1\nGeneral The organization shall monitor, measure, analyse and evaluate its environmental performance. The organization shall determine:a) what needs to be monitored and measured; b) the methods for monitoring, measurement, analysis and evaluation, as applicable, to ensure valid results; c) the criteria against which the organization will evaluate its environmental performance, and appropriate indicators; d) when the monitoring and measuring shall be performed; e) when the results from monitoring and measurement shall be analysed and evaluated. The organization shall ensure that calibrated or verified monitoring and measurement equipment is used and maintained, as appropriate. The organization shall evaluate its environmental performance and the effectiveness of the environmental management system. The organization shall communicate relevant environmental performance information both internally and externally,

In [ ]:
costom_template = PromptTemplate(
    template="""
You are a highly knowledgeable assistant specializing in the JRC assessment framework (JRC code of conduct) and ISO standards (50001, 14001, 30134). 
Your task is to generate a developer-focused framework by interpreting relevant chunks from the provided content. Follow these strict instructions:




1. **Focus on Developer Needs**:
    - Provide actionable technical requirements for developers.
    - Ensure the requirements are framed in plain, developer-friendly language.

2. **Structure**:
    - Include clear headings for each requirement category.
    - Break down requirements into:
        - Title
        - Technical Details (if stated in chunk) - 
                The technical details should be compliant with ISO standard directives part 2 chapter 7. Follow these rules:
                1. For mandatory provisions, use "shall." Example: "The system shall provide a log of all user activities."
                2. For advisory provisions, use "should." Example: "The interface should include a dark mode option for improved user comfort."
                3. For permissible actions, use "may." Example: "Users may configure their own notification preferences."
            4. For statements about potential or capability, use "can." Example: "The system can handle up to 10,000 simultaneous connections."
        - Evidence Required (if stated in chunk)
        - Reference Frameworks (if applicable) and Source (ISO/JRC section)
        - Type of requirement (mandatory, non-mandatory (optional), etc. for iso and WHole datacenter, new build, existing building, etc. and importance value for JRC)
        

3. **Comprehensiveness**:
    - Use **all relevant chunks** to create the framework, even if their relevance is marginal.
    -Mention which chunks you did not use and why.
    - Summarize overlapping or repetitive chunks to avoid redundancy but retain all key details.

4. **Avoid Assumptions**:
    - Only use information explicitly found in the chunks.
    - Do not introduce new concepts or requirements not in the source content.

5. **Formatting**:
    - Present the framework in a structured markdown format.
    - Use numbered sections and subsections for easy navigation.
    

6. **Metadata Inclusion**:
    - For each requirement, include references such as ISO/JRC standards, specific practice numbers, and evidence.

**Context**:
These chunks are sorted by relevance using the RRF method and pertain to the query.

**Chunks**:
{chunks}

**Question**:
{question}

**Answer**:
Provide a comprehensive developer-oriented framework meeting the above criteria.
"""
)


In [ ]:
updated_rephrase_question = PromptTemplate(template= """
You are tasked with rephrasing a user query for dense and sparse search on ISO and JRC documents. 
A good query should maximize precision and recall by focusing on relevant terms, avoiding conversational or reasoning context, and leveraging domain-specific language. 
Your goal is to create a concise, precise, and search-optimized query that retrieves relevant content effectively.



**User Query**:
{user_query}

**Instructions**:
1. Retain the meaning, context, and intent of the original query while simplifying it. Do not reduce it to a list of disconnected keywords.
2. Identify critical keywords or concepts in the user query (e.g., "storage,", "capacity planning") and use them to construct a concise, structured query.
3. Write the rephrased query in the form of meaningful, concise phrases or sentences. Avoid generating disconnected terms or generic lists.
4. Expand the query with synonyms and related terms relevant to the domain (e.g., "storage systems," "capacity planning," "data protection") to improve hybrid search recall.
5. Tailor the query to support both dense (semantic) and sparse (keyword-based) retrieval by including specific phrases that enhance matching.
6. Avoid over-simplification or redundancy. Ensure the rephrased query is meaningful and aligned with the original user query.

**Example Rephrasing**:
User Query: "what should I do for storage management in my new datacenter?"
Bad Output: "Datacenter storage, ISO compliance, best practices."
Bad output 2: "Best practices and ISO guidelines for datacenter storage, ISO guidelines, new facility."
bad output 3: "best pracitses and guidelenes Datacenter storage management; ISO guidelines, best practices."
Good Output: "Best practices and ISO guidelines for datacenter storage management in new facilities."

**Search Query**:                                 
Please provide a concise and precise search query based on the above instructions.                                   
                                   
         
                                   """)


def rephrase_query(user_query):
   new_quest = updated_rephrase_question.format(user_query=user_query)
   new_query = Chat_llm.invoke(new_quest)
    
   return new_query

In [80]:
def updated_query_remake(prompt, iterations):
    all_repharased_data = []
    
    for i in range(iterations):
        new_query = rephrase_query(prompt)
        all_repharased_data.append(new_query.content)
        
        
        
    return all_repharased_data


In [82]:
Query = "how should I define kpis for a new datacenter"
query_rephrased = rephrase_query(Query)
print(query_rephrased.content)

"Key performance indicators (KPIs) and best practices for datacenter performance management and monitoring"


In [51]:

Guidelines and best practices for storage system upgrades when retrofitting a datacenter, including capacity planning and data protection
Guidelines and strategies for storage solutions when retrofitting datacenters, including ISO compliance and capacity planning.

SyntaxError: invalid syntax (1227779763.py, line 1)

In [15]:
def fetch_chunks_rrf_sorted(user_question, limitN):
    dense_results = merged_dense_for_RRF(user_question, limitN)
    print(dense_results)
    sparse_results = merged_sparse_for_RRF(user_question, limitN)
    print(sparse_results)
    fused_results = reciprocal_rank_fusion(dense_results, sparse_results, k=60)
    Nodes_rrf_back = Fetch_chunks_back(fused_results)
    return Nodes_rrf_back



In [ ]:

def conversational_system(query):
   
    # Fetch context from the fused results
    new_query = rephrase_query(query)
    context = fetch_chunks_rrf_sorted(new_query.content, 25)
    print("Query used: ", new_query)


    
    custom_prompt = costom_template.format(chunks=context[:15], question=query)
    response = Chat_llm.invoke(custom_prompt)
    
    # Add to memory
    
    print(len(context[:15]))
    
    print("Your query: ", query)
    return response , context





In [19]:
def ask_rag(user_input):
    start_time = time.time()  # Measure processing time
    bot_response, chunks_used = conversational_system(user_input)
    end_time = time.time()
    time_taken = end_time - start_time
    return bot_response.content, chunks_used, time_taken



In [25]:
# query = input("User: ")
query= "How should KPIs be defined, measured, and reported for a new datacenter Im designing?"

In [27]:
response, chunks_used, time_taken = ask_rag(query)

points=[ScoredPoint(id='ba2fb31b-0f08-4c22-a523-53f8b0daeecf', version=0, score=0.6395190077703651, payload={'content': 'A.1 General\nThe KPIs of the ISO/IEC 30134 series are intended to promote the reduction of the environmental impacts of a data centre. However, availability objectives and location can constrain the achievable improvements. There should be no “judgement” of an actual KPI without attaching to the data point a description of availability objectives and the location/climate of the data centre under consideration. KPIs should be used to help data centre managers to improve the data centre’s resource usage effectiveness or efficiency by comparing and assessing improvement options.', 'section_id': 'A.1', 'parent_id': 'A', 'parent_title': 'A non-compliance is not necessarily elevated to a nonconformity if, for example, it is identified and corrected by the environmental management system processes. Compliance-related nonconformities need to be corrected, even if those nonco

In [23]:
print(response)

# Developer-Focused Framework for Energy Performance Improvement

This document translates the JRC assessment framework and ISO standards related to energy management into actionable developer-oriented requirements. Each section below is organized by requirement categories with clearly defined technical details, evidence requirements, reference frameworks, sources, and requirement types.

## 1. Risk and Opportunity Management

### 1.1 Risk and Opportunity Identification and Management
- **Technical Details**: 
  - The organization shall plan actions to address risks and opportunities (ISO 50001, section 6.1.2).
  - The organization shall consider risks and opportunities related to compliance obligations, environmental conditions, and needs of interested parties (ISO 14001, section A.6.1.1).
- **Evidence Required**: None specified.
- **Reference Frameworks**: ISO 50001, section 6.1.2; ISO 14001, section A.6.1.1.
- **Type of Requirement**: Mandatory for energy management systems.

## 2. 

In [24]:
print("Time taken: ", time_taken)

Time taken:  45.10594058036804
